In [2]:
import pandas as pd
import bokeh.io
path_to_incubations = "../lab_work/sessions/incubations/"
from GCPDFReader.server.data_viz import create_co2_chart, create_methane_chart, get_methane_df, create_methane_incubation_length_chart, create_co2_incubation_length_chart, create_2mL_methane_incubation_length_chart, create_2mL_co2_incubation_length_chart, create_40mL_methane_incubation_length_chart, create_40mL_co2_incubation_length_chart, create_40mL_methane_chart, create_40mL_co2_chart, create_2mL_methane_chart, create_2mL_co2_chart
from GCPDFReader.server.extract_incubation_data import extract_and_process_incubation_data
from GCPDFReader.server.transform import process_sample_data
from bokeh.plotting import output_file, save

bokeh.io.output_notebook()

csv_path = "csvs/tidied_1694731480.csv"
data_df = pd.read_csv(f"GCPDFReader/{csv_path}")
incubation_df = extract_and_process_incubation_data(path_to_incubations)

df = process_sample_data(data_df, incubation_df)


plot = create_co2_chart(df)
# output_file(filename="GCPDFREADER/figures/co2_conc.html", title="CO2 Concentration v Organic Matter: Salt")
# save(plot)
bokeh.io.show(plot)

Loading BokehJS ...

In [2]:
CO2_df = df[(df['peak_compound'] == 'CO2') & (df['is_std'] == False) & (df['sample_id']!="DROP_ME")]
CO2_df[['upper', 'lower', 'calculated_conc']]

,upper,lower,calculated_conc
1,2819.647757,2712.647757,2766.147757
24,2991.238337,2884.238337,2937.738337
31,1544.650993,1437.650993,1491.150993
33,3904.651607,3797.651607,3851.151607
37,97397.825000,97290.825000,97344.325000
...,...,...,...
769,8446.093022,8339.093022,8392.593022
777,253776.250000,253669.250000,253722.750000
779,15568.047845,15461.047845,15514.547845
781,170940.000000,170833.000000,170886.500000


In [2]:
plot = create_methane_chart(df)
bokeh.io.show(plot)


In [2]:
plot = create_methane_incubation_length_chart(df)
bokeh.io.show(plot)

In [3]:
plot = create_co2_incubation_length_chart(df)
bokeh.io.show(plot)

In [ ]:
temp = np.array([[142828.00, 20000.00],
[143498.00, 20000.00],
[141214.00, 20000.00],
[140473.00, 20000.00],
[142588.00, 20000.00],
[97.75, 2.18],
[57.81, 2.18],
[69.16, 2.18],
[88.26, 2.18]])
G = np.vstack([np.ones(temp.shape[0]), temp[:, 0]]).T
d = temp[:, 1]
m = np.linalg.lstsq(G, d)[0]
print(m)
test_areas = np.array([100, 1e3, 1e4, 1e5])
cov = np.linalg.inv(G.T @ G)
testG = np.vstack([np.ones(test_areas.shape[0]), test_areas]).T
print(np.sqrt(np.diag(testG @ cov @ testG.T))) 

In [4]:
hold = create_methane_incubation_length_chart(df)
bokeh.io.show(hold)

In [ ]:
hold = create_co2_incubation_length_chart(df)
bokeh.io.show(hold)

In [6]:
hold = create_methane_incubation_length_chart(df)
bokeh.io.show(hold)

In [5]:
hold = create_40mL_methane_incubation_length_chart(df)
bokeh.io.show(hold)

In [ ]:
hold = create_40mL_co2_incubation_length_chart(df)
bokeh.io.show(hold)

In [ ]:
hold = create_40mL_methane_incubation_length_chart(df)
bokeh.io.show(hold)

In [7]:
plot = create_2mL_methane_chart(df)
bokeh.io.show(plot)

In [ ]:
plot = create_40mL_methane_chart(df)
bokeh.io.show(plot)

In [ ]:
plot = create_2mL_co2_chart(df)
bokeh.io.show(plot)

In [ ]:
plot = create_40mL_co2_chart(df)
bokeh.io.show(plot)

In [ ]:
import bokeh.io
from bokeh import plotting
from bokeh.io import export_png
from bokeh.models import ColumnDataSource, ColorBar, LinearColorMapper, Span, CategoricalColorMapper
from bokeh.palettes import Spectral6, Inferno256
from bokeh.transform import jitter
from GCPDFReader.server.constants import COMPOUNDS

import pandas as pd
import numpy as np


In [ ]:
def time_v_conc_connect_time_points(title, df, atmospheric_conc):
    incubation_length_max = int(df["incubation_length"].max())
    wet_df = ColumnDataSource(df[df["treatment"]=="Wet"])
    dry_df = ColumnDataSource(df[df["treatment"]=="Dry"])
    
    
    clr_mapper = CategoricalColorMapper(palette=["red", "blue", "green", "purple"], factors=["1:0", "16:1", "1:1", "1:5"])


    TOOLTIPS = [
    ("sample_id", "@sample_id"),
    ("Measured Concentration", "@calculated_conc")
        ]
    
    p = plotting.figure(
        title=title,
        x_axis_label = "Incubation Length (Days)",
        y_axis_label = "Measured Concentration(ppm)",
        y_axis_type="log",
        y_range=[10e-2, 10e5],
        x_range=[30, incubation_length_max+10],
        toolbar_location='above',
        tooltips = TOOLTIPS,
        sizing_mode="stretch_width"
    )
    p.triangle(
        source=wet_df,
        x = jitter("incubation_length", width=0.5, range=p.x_range),
        y = "calculated_conc",
        color={"field":"salt_ratio", "transform":clr_mapper},
        size=10,
        legend_label="Wet Treatment"
        )
    p.plus(
        source=dry_df,
                x = jitter("incubation_length", width=0.5, range=p.x_range),
        y = "calculated_conc",
        color={"field":"salt_ratio", "transform":clr_mapper},
        size=10,
        legend_label="Dry Treatment"
        )
    
    for id in df['sample_id'].unique():
        print(id)
        sub_df = df[df['sample_id'] == id]
        print(sub_df.shape)
        p.line(
            source = sub_df,
            x = "incubation_length",
            y="calculated_conc"
        )
    if atmospheric_conc:
        atm_conc_line = Span(location=atmospheric_conc, dimension='width', line_color='red', line_width=2)
        p.add_layout(atm_conc_line)
   
    p.add_layout(bokeh.models.Legend(), "above")
    p.legend.click_policy = "hide"
    return p
    




In [ ]:
df['sample_id'].unique()

In [ ]:

methane_df = get_methane_df(df)
forty_mL_df = methane_df[methane_df["Sample_Name"].str.startswith("40ML")]
hold = time_v_conc_connect_time_points('something', forty_mL_df, 440)
bokeh.io.show(hold)

In [ ]:
forty_mL_df.shape